In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import pandas as pd
import duck_db_helper
import duckdb
from sklearn.model_selection import train_test_split

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/feature_generation/feature_generation.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()
df = df_list[0]

The selected model is a Multiple Linear Regression model. We want to analyse the features that affect a players performance as well as predict how a player will perform. The train/test split stratify the Player name so that each player is represented in both train and test set. 

In [4]:
X = df.drop('total_points', axis=1)  # Features (excluding the target variable)
y = df['total_points']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df['name'], random_state=42)
# readd the target to the data
X_train['total_points'] = y_train
X_test['total_points'] = y_test

In [6]:
conn = duckdb.connect('../data/feature_generation/train.db')
duck_db_helper.create_table('train_feature_1', X_train, conn)
conn.close()
conn = duckdb.connect('../data/feature_generation/test.db')
duck_db_helper.create_table('test_feature_1', X_test, conn)
conn.close()